In [4]:
import os, sys, re, ast
import pandas as pd
import numpy as np
import bct

In [126]:
sv_df = pd.read_csv("sv_shared_pubs.csv")
gs_df = pd.read_csv("shared_pubs.csv")

In [127]:
print(sv_df.head())

                                               Title  \
0  Chronic multiscale resolution of mouse brain n...   
1  Water-Reaching Platform for Longitudinal Asses...   
2  Multiscale imaging informs translational mouse...   
3  Towards a Visualizable, De-identified Syntheti...   
4  Altered cortical processing of sensory input i...   

                                     Authors  
0   ['Timothy H. Murphy', 'Lynn A. Raymond']  
1   ['Timothy H. Murphy', 'Lynn A. Raymond']  
2  ['Timothy H. Murphy', 'Jeffrey M. LeDue']  
3      ['Timothy H. Murphy', 'Helge Rhodin']  
4   ['Timothy H. Murphy', 'Lynn A. Raymond']  


In [130]:
def standardize_title(title:str) -> str:
    '''return clean cleaned, lowercase string'''

    title = "".join(
        re.sub(r'[^\w ]', '', #replace anything that is not a letter with ""
               title.replace("<sup>","") #remove tags (mostly in scival data)
               .replace("</sup>","")
               .replace("<inf>","")
               .replace("</inf>","")
               .lower(), #lower all letters
               ) 
            .split(" "))
    return title

In [131]:
#clean titles
sv_df["Standard Titles"] = pd.DataFrame([standardize_title(title) for title in sv_df["Title"].tolist()])
gs_df["Standard Titles"] = pd.DataFrame([standardize_title(title) for title in gs_df["Title"].tolist()])

In [132]:
print(sv_df.head())

                                               Title  \
0  Chronic multiscale resolution of mouse brain n...   
1  Water-Reaching Platform for Longitudinal Asses...   
2  Multiscale imaging informs translational mouse...   
3  Towards a Visualizable, De-identified Syntheti...   
4  Altered cortical processing of sensory input i...   

                                     Authors  \
0   ['Timothy H. Murphy', 'Lynn A. Raymond']   
1   ['Timothy H. Murphy', 'Lynn A. Raymond']   
2  ['Timothy H. Murphy', 'Jeffrey M. LeDue']   
3      ['Timothy H. Murphy', 'Helge Rhodin']   
4   ['Timothy H. Murphy', 'Lynn A. Raymond']   

                                     Standard Titles  
0  chronicmultiscaleresolutionofmousebrainnetwork...  
1  waterreachingplatformforlongitudinalassessment...  
2  multiscaleimaginginformstranslationalmousemode...  
3  towardsavisualizabledeidentifiedsyntheticbioma...  
4  alteredcorticalprocessingofsensoryinputinhunti...  


In [133]:
#turn into a dictionary of keys with coauthors as values

# s = [ast.literal_eval(coauthors) for coauthors in sv_df["Authors"].tolist()]
# print(s)

# s = [ast.literal_eval(coauthors) for coauthors in sv_df["Authors"].tolist()]

sv_dict = sv_df.set_index('Standard Titles').T.to_dict('list')
gs_dict = gs_df.set_index('Standard Titles').T.to_dict('list')

/var/folders/47/6bm96nxd0ks5x370bsvkpghh0000gn/T/ipykernel_88020/3357728429.py:9: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  gs_dict = gs_df.set_index('Standard Titles').T.to_dict('list')


In [134]:
sv_only = list(set(sv_dict.keys()) - set(gs_dict.keys()))
gs_only = list(set(gs_dict.keys()) - set(sv_dict.keys()))
shared_titles = list(set(sv_dict.keys()).intersection(gs_dict.keys()))
print("total sv publications:", len(sv_dict.keys()))
print("total gs publications:", len(gs_dict.keys()))
print()
print("sv only publications:",len(sv_only))
print("gs only publications:",len(gs_only))
print("shared publications:",len(shared_titles))

total sv publications: 360
total gs publications: 477

sv only publications: 64
gs only publications: 181
shared publications: 296


In [137]:
fill_space = lambda x: [""]*(len(shared_titles)-len(x))
out_df = pd.DataFrame(
    {
        "GS ONLY publications":gs_only 
                                + fill_space(gs_only),
        "GS ONLY title": [gs_dict[title][0] for title in gs_only] 
                                + fill_space(gs_only),
        "GS ONLY coauthors": [gs_dict[title][1] for title in gs_only] 
                                + fill_space(gs_only),
        "SV ONLY publications": sv_only 
                                + fill_space(sv_only),
        "SV ONLY title": [sv_dict[title][0] for title in sv_only] 
                                + fill_space(sv_only),
        "SV ONLY coauthors": [sv_dict[title][1] for title in sv_only] 
                                + fill_space(sv_only),
        "shared publications": shared_titles,
        "shared gs title": [gs_dict[title][0] for title in shared_titles],
        "shared sv title": [sv_dict[title][0] for title in shared_titles],
        "shared gs coauthors": [gs_dict[title][1] for title in shared_titles],
        "shared sv coauthors": [sv_dict[title][1] for title in shared_titles],
    })

In [138]:
out_df.to_csv("check1.csv",index=False)